In [2]:
import pandas as pd
import numpy as np
import xlrd
from scipy.stats import norm
import openpyxl
import sys, math
import matplotlib.pyplot as plt

file_path = "PDpettitt.xlsx"
data = pd.read_excel(file_path)

data.head()

,LID,z
0,288,20
1,289,22
2,290,22
3,291,22
4,292,20


In [19]:
data1 = xlrd.open_workbook(file_path)

table = data1.sheets()[0]

nrows = table.nrows #行数

ncols = table.ncols #列数
print(nrows)
print(ncols)
print(table.row_values(1))

111403
32
[288.0, 20.202, 60.6061, 80.8081, 40.404, 20.202, 60.6061, 40.404, 80.8081, 20.202, 61.7284, 41.1523, 61.7284, 20.5761, 61.7284, 41.1523, 41.1523, 20.5761, 41.1523, 20.5761, 60.6061, 20.202, 40.404, 40.404, 20.202, 40.404, 20.202, 40.404, 20.202, 40.404, 7.5584999999999996, 7.5584999999999996]


In [20]:
def pettitt(X):
    # second one
    U = []; s = 0
    U.append(sum(np.sign(X[0] - np.array(X[:]))))
    for t in range(1, len(X)):
        v_t = sum(np.sign(X[t] - np.array(X[:])))
        U.append(U[t-1] + v_t)
    Kt = max(np.abs(U))
    max_idx = list(np.abs(U)).index(max(np.abs(U)))
    return U, max_idx, Kt

def plot(X, Ut, Kt, max_idx):
    fig = plt.figure(figsize = (9, 6))
    plt.subplots_adjust(left=0.1, right=0.9, top=0.98, bottom=0.05, wspace=None, hspace=0.5)
    axes = []
    for i in range(2):
        ax = fig.add_subplot(2, 1, i + 1)
        axes.append(ax)
    # 原始数据
    axes[0].plot(range(1, len(X) + 1), X)
    axes[0].plot([max_idx + 1, max_idx + 1], [min(X) - 0.5, max(X) + 0.5], linestyle = "--", c = "r")
    # axes[0].set_xlabel("Index")
    axes[0].set_ylabel("X")
    axes[0].set_xlim((0 - 1, len(X) + 1))
    axes[0].set_ylim((min(X) - 0.5, max(X) + 0.5))

    axes[1].plot(range(1, len(Ut) + 1), np.abs(Ut), label = "Ut")
    axes[1].plot([max_idx + 1, max_idx + 1], [0, Kt], linestyle = "--", c = "r")

    Kt_005, Kt_005_n = CalKtByAlpha(0.05, len(X))
    axes[1].plot([1, len(Ut)], [Kt_005, Kt_005], linestyle = "-.", c = "r")
    axes[1].plot([1, len(Ut)], [Kt_005_n, Kt_005_n], linestyle = "-.", c = "r", label = "p=0.05")

    Kt_001, Kt_001_n = CalKtByAlpha(0.01, len(X))
    axes[1].plot([1, len(Ut)], [Kt_001, Kt_001], linestyle = "--", c = "b")
    axes[1].plot([1, len(Ut)], [Kt_001_n, Kt_001_n], linestyle = "--", c = "b", label = "p=0.01")

    Kt_0001, Kt_0001_n = CalKtByAlpha(0.001, len(X))
    axes[1].plot([1, len(Ut)], [Kt_0001, Kt_0001], linestyle = ":", c = "g")
    axes[1].plot([1, len(Ut)], [Kt_0001_n, Kt_0001_n], linestyle = ":", c = "g", label = "p=0.001")

    axes[1].set_xlim((0 - 1, len(X) + 1))
    axes[1].set_ylabel("Ut")
    plt.legend(loc = "upper left")
    plt.show()

def CalP(Kt, n):
    return 2.0 * math.exp((-6.0 * Kt ** 2) / float(n ** 3 + n ** 2))

def CalKtByAlpha(alpha, n):
    Kt = math.sqrt(np.log(alpha / 2.0) / -6.0 * (n ** 3 + n ** 2))
    return Kt, -Kt

In [21]:
listresult = [['LID','t']]#声明列表中的头
listx = ["1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

for i in range(1,nrows-10):
    #nrows-1
    test= table.row_values(i) #某一行数据
    
    alpha = 0.05
    # 	data = pd.read_csv("data_random.csv")
    Year = list(listx)
    X = list(test[1:])
#     print(X)

    Ut, max_idx, Kt = pettitt(X)
#     print("The %dth sample is change point. Kt=%.2f, Year:%d, p=%.5f" % (max_idx + 1, Kt, Year[max_idx], CalP(Kt, len(X))))
#     if CalP(Kt, len(X)) < alpha:
#         print("The %dth sample is change point. Year:%d, p=%.9f" % (max_idx + 1, Year[max_idx], CalP(Kt, len(X))))
#     else:
#         print("Does not satisfy the significance level! P(Kt) = %.5f" % CalP(Kt, len(X)))
    print(test[0],Year[max_idx])

    listresult.append([test[0],Year[max_idx]])
    
#*****************存储到excel表格中
wb=openpyxl.Workbook()#创建一个工作薄 
sheet=wb.active#创建一个工作表
sheet.title='year'#为sheet页起名
for i in listresult:
    sheet.append(i)#将列表中的内容存储到文件
wb.save('PDtubianyearpettitt.xlsx')#保存

288.0 2010
289.0 2012
290.0 2012
291.0 2012
292.0 2010
293.0 2011
294.0 2009
295.0 2010
682.0 2009
712.0 2010
713.0 2013
714.0 2012
715.0 2012
716.0 2011
717.0 2009
718.0 2010
719.0 2011
720.0 2012
721.0 2005
1106.0 1998
1107.0 2004
1136.0 1993
1137.0 2012
1138.0 2012
1139.0 2011
1140.0 2011
1141.0 2012
1142.0 2011
1143.0 2009
1144.0 2009
1145.0 2011
1146.0 2011
1531.0 2007
1532.0 2009
1533.0 2009
1559.0 2009
1560.0 2001
1561.0 2012
1562.0 2012
1563.0 2009
1564.0 2011
1565.0 2003
1566.0 2012
1567.0 2012
1568.0 2012
1569.0 2011
1570.0 2011
1571.0 2011
1572.0 2003
1956.0 2013
1957.0 2013
1958.0 2009
1959.0 2009
1960.0 2009
1983.0 2019
1984.0 2012
1985.0 2012
1986.0 2012
1987.0 2012
1988.0 2004
1989.0 2012
1990.0 2012
1991.0 2012
1992.0 2009
1993.0 2007
1994.0 2011
1995.0 2011
1996.0 2009
1997.0 2005
1998.0 2016
1999.0 1993
2000.0 2003
2382.0 2007
2383.0 2009
2384.0 2009
2385.0 2009
2386.0 2012
2389.0 2007
2390.0 2007
2391.0 2006
2392.0 2011
2393.0 2009
2398.0 2014
2399.0 2009
2400.0 2007

8753.0 2009
8754.0 2009
8755.0 2009
8756.0 2016
8757.0 2008
8758.0 2009
8759.0 2010
8760.0 2011
8761.0 2004
8762.0 2011
8763.0 2011
8764.0 2011
8765.0 2011
8766.0 2007
8767.0 2011
8768.0 2012
8769.0 2009
8770.0 2011
8771.0 2011
8772.0 2011
8773.0 2012
8774.0 2002
8775.0 2011
8776.0 2009
8777.0 2009
8778.0 1998
8779.0 2009
8780.0 2009
8781.0 2012
8782.0 2012
8783.0 2012
8784.0 1999
8785.0 2011
8786.0 2004
8787.0 1998
8788.0 2005
8789.0 1994
9175.0 2008
9176.0 2009
9177.0 2011
9178.0 2008
9179.0 2009
9180.0 2009
9181.0 2009
9182.0 2007
9183.0 2008
9184.0 2011
9185.0 2012
9186.0 2004
9187.0 2011
9188.0 2003
9189.0 2009
9190.0 2007
9191.0 2006
9192.0 2010
9193.0 2007
9194.0 2003
9195.0 2009
9196.0 2011
9197.0 2011
9198.0 2012
9199.0 2012
9200.0 1996
9201.0 2009
9202.0 2011
9203.0 1995
9204.0 2009
9205.0 2012
9206.0 1995
9207.0 2000
9208.0 2012
9209.0 2012
9210.0 2003
9211.0 2011
9212.0 1995
9213.0 1995
9599.0 2007
9600.0 2007
9601.0 2009
9602.0 2009
9603.0 2011
9604.0 2009
9605.0 2009
9606

16402.0 2003
16403.0 2009
16404.0 1998
16405.0 1998
16406.0 2011
16407.0 2011
16408.0 2009
16409.0 2011
16410.0 2004
16411.0 1999
16412.0 2011
16413.0 2000
16414.0 1999
16415.0 2011
16416.0 1995
16795.0 2018
16796.0 2003
16797.0 1999
16798.0 2005
16799.0 2011
16800.0 2016
16801.0 2009
16802.0 1996
16803.0 2011
16804.0 2011
16805.0 2009
16806.0 2011
16807.0 1995
16808.0 2011
16809.0 1995
16810.0 1995
16811.0 2011
16812.0 2002
16813.0 2011
16814.0 1995
16815.0 2011
16816.0 2011
16817.0 2011
16818.0 2011
16819.0 2011
16820.0 2004
16821.0 2011
16822.0 2009
16823.0 2008
16824.0 2006
16825.0 2003
16826.0 2004
16827.0 1999
16828.0 2009
16829.0 2014
16830.0 1998
16831.0 1992
16832.0 2000
16833.0 1995
16834.0 1995
16835.0 1999
16836.0 2016
16837.0 2008
16838.0 2001
16839.0 2012
16840.0 2011
16841.0 2004
17218.0 2009
17219.0 2000
17220.0 2010
17221.0 2004
17222.0 2016
17223.0 2011
17224.0 2010
17225.0 1998
17226.0 2009
17227.0 2010
17228.0 2011
17229.0 1996
17230.0 2012
17231.0 2010
17232.0 2000

22769.0 2003
22770.0 1995
22771.0 2009
22772.0 2003
22773.0 1995
22774.0 2011
22775.0 1999
22776.0 2004
22777.0 1998
22778.0 1995
22779.0 2001
22780.0 2010
22781.0 2010
22782.0 1995
22783.0 2009
22784.0 2004
22785.0 2009
22786.0 1995
22787.0 2016
23163.0 2005
23164.0 2011
23165.0 2018
23166.0 2011
23167.0 2000
23168.0 1999
23169.0 2011
23170.0 2011
23171.0 1994
23172.0 2009
23173.0 2016
23174.0 2011
23175.0 1995
23176.0 2003
23177.0 2004
23178.0 2004
23179.0 2011
23180.0 2010
23181.0 2009
23182.0 2011
23183.0 2011
23184.0 2003
23185.0 2009
23186.0 2009
23187.0 1999
23188.0 2003
23189.0 2018
23190.0 1996
23191.0 2002
23192.0 2011
23193.0 2009
23194.0 2009
23195.0 1992
23196.0 2009
23197.0 1995
23198.0 1995
23199.0 2012
23200.0 2009
23201.0 2003
23202.0 1995
23203.0 1995
23204.0 2010
23205.0 2010
23206.0 1995
23207.0 2002
23208.0 2012
23209.0 2002
23210.0 2009
23211.0 2009
23212.0 1995
23587.0 1994
23588.0 2004
23589.0 2011
23590.0 2000
23591.0 2009
23592.0 2009
23593.0 2004
23594.0 2010

29571.0 2009
29572.0 2008
29573.0 1996
29574.0 2004
29575.0 2015
29576.0 2007
29577.0 1996
29578.0 2009
29579.0 2009
29580.0 2009
29581.0 2014
29582.0 2015
29583.0 2003
29584.0 2008
29585.0 2011
29586.0 2009
29587.0 1999
29588.0 2005
29599.0 2003
29600.0 2011
29601.0 2011
29961.0 2016
29962.0 2003
29963.0 2011
29964.0 2011
29965.0 2007
29966.0 2011
29967.0 2010
29968.0 2001
29969.0 2009
29970.0 2011
29971.0 2009
29972.0 2011
29973.0 2011
29974.0 2011
29975.0 2011
29976.0 1998
29977.0 2011
29978.0 2009
29979.0 1996
29980.0 2004
29981.0 1996
29982.0 2004
29983.0 2003
29984.0 2007
29985.0 1998
29986.0 2008
29987.0 1996
29988.0 2011
29989.0 2013
29990.0 2009
29991.0 2009
29992.0 2005
29993.0 1995
29994.0 2009
29995.0 1996
29996.0 2000
29997.0 2006
29998.0 2011
29999.0 2004
30000.0 2005
30001.0 2004
30002.0 1995
30003.0 2009
30004.0 2009
30005.0 2008
30006.0 2004
30007.0 2010
30008.0 1999
30009.0 1996
30010.0 1996
30011.0 2009
30012.0 2011
30013.0 2011
30014.0 1999
30023.0 1999
30024.0 1997

...

281000.0 1999
281001.0 2014
281002.0 2014
281003.0 2013
281004.0 2009
281005.0 2014
281006.0 2014
281007.0 2014
281008.0 2009
281009.0 2000
281010.0 1999
281011.0 1998
281012.0 2005
281013.0 1999
281014.0 2003
281015.0 2012
281016.0 1998
281017.0 1998
281018.0 2009
281019.0 2003
281020.0 1998
281021.0 2014
281022.0 2015
281023.0 2014
281024.0 2003
281025.0 2014
281026.0 2014
281027.0 2012
281028.0 2003
281029.0 2014
281030.0 2014
281031.0 2013
281032.0 2014
281033.0 2014
281034.0 2014
281035.0 2003
281036.0 2014
281037.0 2005
281038.0 2014
281039.0 2014
281040.0 2012
281041.0 2003
281042.0 2014
281043.0 1998
281044.0 2007
281045.0 2014
281046.0 2008
281047.0 2003
281048.0 2003
281049.0 2003
281050.0 2010
281051.0 2014
281052.0 1998
281053.0 2014
281054.0 2014
281055.0 2014
281056.0 2014
281057.0 2014
281058.0 2014
281059.0 2014
281060.0 2014
281061.0 2012
281062.0 2014
281063.0 2014
281064.0 2013
281065.0 1999
281066.0 2014
281067.0 2014
281068.0 2012
281069.0 2014
281070.0 2014
281071

282711.0 2013
282712.0 2005
282713.0 2003
282714.0 2013
282715.0 2013
282716.0 2013
282717.0 2011
282718.0 2013
282719.0 2011
282720.0 2009
282721.0 2011
282722.0 2007
282723.0 2007
282724.0 2009
282725.0 2014
282726.0 2009
282727.0 2003
282728.0 2009
282729.0 2013
282730.0 2013
282731.0 2014
282732.0 2003
282733.0 2003
282734.0 2013
282735.0 2013
282736.0 2007
282737.0 2008
282738.0 2014
282739.0 2003
282740.0 2003
282741.0 2002
282742.0 1998
282743.0 1999
282744.0 2017
282745.0 2007
282746.0 2013
282747.0 2014
282748.0 2014
282749.0 2014
282750.0 2012
282751.0 2014
282752.0 2014
282753.0 2014
282754.0 2013
282755.0 2013
282756.0 2013
282757.0 2013
282758.0 2013
282759.0 2013
282760.0 2013
282761.0 2013
282762.0 2012
282763.0 2009
282764.0 2007
282765.0 2003
282766.0 1999
282767.0 1999
282768.0 2003
282769.0 2013
282770.0 2014
282771.0 2014
282772.0 2014
282773.0 2014
282774.0 2014
282775.0 2015
282776.0 2013
282777.0 2014
282778.0 2004
282779.0 2009
282780.0 2013
282781.0 2004
282782

284799.0 2009
284800.0 1998
284801.0 2018
284802.0 1996
284803.0 2014
284804.0 2009
284805.0 2005
284806.0 1998
284807.0 1998
284808.0 1999
284809.0 1998
284810.0 2014
284811.0 2018
284812.0 2014
284813.0 2007
284814.0 2014
284815.0 2009
284816.0 2014
284817.0 2014
284818.0 2003
284819.0 2009
284820.0 2009
284821.0 2003
284822.0 2003
284823.0 2003
284824.0 2009
284825.0 2009
284826.0 2009
284827.0 2009
284828.0 2009
284829.0 2014
284830.0 2014
284831.0 2016
284835.0 2014
284836.0 2003
284837.0 2003
284838.0 2003
284839.0 2003
284840.0 2003
284841.0 2003
284842.0 2003
284843.0 2014
284844.0 2014
284845.0 2014
284846.0 2007
284847.0 2014
284848.0 2003
284849.0 2009
284850.0 2003
284851.0 2003
284852.0 2003
284853.0 1999
284854.0 2003
284855.0 2003
284856.0 2003
284857.0 2003
284858.0 2003
284859.0 2014
284860.0 2014
284861.0 2014
284862.0 2005
284863.0 2014
284864.0 2014
284865.0 1999
284866.0 1999
284867.0 1997
284868.0 2003
284869.0 2003
284870.0 2014
284871.0 2014
284872.0 2008
284873

286896.0 1998
286897.0 1998
286898.0 1998
286899.0 1999
286900.0 1999
286901.0 1999
286902.0 2005
286903.0 1999
286904.0 2005
286905.0 2005
286906.0 2009
286907.0 2003
286908.0 1999
286909.0 2003
286910.0 2000
286911.0 2003
286912.0 2003
286913.0 2003
286914.0 2003
286915.0 2014
286916.0 1999
286917.0 2014
286918.0 2009
286919.0 2009
286920.0 2009
286921.0 2007
286922.0 2003
286923.0 2003
286924.0 2003
286925.0 2002
286926.0 2016
286927.0 1996
286928.0 2008
286929.0 2005
286930.0 2014
286931.0 2018
286932.0 2014
286933.0 2003
286934.0 2014
286935.0 2014
286936.0 2014
286937.0 2003
286938.0 2014
286939.0 2013
286940.0 2009
286941.0 2008
286942.0 2005
286943.0 2004
286944.0 2003
286945.0 2014
286946.0 2003
286947.0 2003
286948.0 2003
286949.0 2003
286950.0 2003
286951.0 2009
286952.0 2003
286953.0 2003
286954.0 2004
286964.0 2005
286965.0 2003
286966.0 2003
286967.0 2003
286968.0 2003
286969.0 1999
286970.0 2003
286971.0 2003
286972.0 2014
286973.0 1999
286974.0 2009
286975.0 2003
286976

289485.0 2009
289486.0 2008
289487.0 2014
289488.0 2014
289489.0 2009
289490.0 2014
289491.0 2003
289492.0 2003
289493.0 2003
289494.0 2003
289495.0 2003
289496.0 2004
289497.0 2004
289527.0 2005
289528.0 2008
289529.0 1999
289530.0 1999
289531.0 1999
289532.0 2003
289549.0 1999
289550.0 1999
289551.0 2014
289552.0 1999
289553.0 1999
289554.0 2002
289555.0 2013
289556.0 2013
289557.0 2014
289558.0 2014
289559.0 2009
289560.0 2013
289561.0 2014
289562.0 2005
289563.0 2004
289564.0 2008
289565.0 1994
289566.0 1996
289567.0 2016
289568.0 1997
289569.0 1998
289570.0 1999
289571.0 1999
289572.0 1999
289573.0 2015
289574.0 1999
289575.0 1999
289576.0 2002
289577.0 1999
289578.0 1999
289579.0 2011
289580.0 2009
289581.0 2002
289582.0 2002
289583.0 2012
289584.0 2012
289585.0 2000
289586.0 2004
289587.0 2001
289588.0 2001
289589.0 2016
289590.0 2014
289591.0 2002
289592.0 2013
289593.0 2001
289594.0 2001
289867.0 1999
289868.0 2003
289869.0 2005
289870.0 2005
289871.0 2005
289872.0 2005
289873